In [1]:
# For Table 5 in the paper.
import pandas as pd

# analyze the exp with non_rt_task
result_dir = "../../real_linux_application/exp_results_non_RT"
schedulers = ["CFS", "FIFO", "RR"]

# set task_info in dataset
result_path = result_dir + "/" + schedulers[-1] + "_result.json"
results_df = pd.read_json(result_path)
results_df = results_df[["task_name", "deadline_ns", "wcet_ns", "priority"]]

for scheduler in schedulers:
    result_path = result_dir + "/" + scheduler + "_result.json"
    result_df = pd.read_json(result_path)
    result_df = result_df[["task_name", "wcrt_ns"]]
    result_df.columns = ["task_name", scheduler + "_wcrt_ns"]
    results_df = pd.merge(results_df, result_df, on="task_name")

# convert to ms
results_df["deadline_ms"] = results_df["deadline_ns"] / 1000000
results_df["wcet_ms"] = results_df["wcet_ns"] / 1000000
results_df["CFS_wcrt_ms"] = results_df["CFS_wcrt_ns"] / 1000000
results_df["FIFO_wcrt_ms"] = results_df["FIFO_wcrt_ns"] / 1000000
results_df["RR_wcrt_ms"] = results_df["RR_wcrt_ns"] / 1000000

# round to 2 decimal places
results_df = results_df.round(1)

# drop ns columns
results_df = results_df.drop(["deadline_ns", "wcet_ns", "CFS_wcrt_ns", "FIFO_wcrt_ns", "RR_wcrt_ns"], axis=1)
results_df

,task_name,priority,deadline_ms,wcet_ms,CFS_wcrt_ms,FIFO_wcrt_ms,RR_wcrt_ms
0,task1,78,100.0,2.8,15.3,4.8,6.3
1,task2,50,1000.0,423.0,912.7,757.4,757.4
2,task3,89,50.0,1.8,14.8,1.9,1.9
3,task4,55,500.0,69.2,167.8,116.4,116.4
4,task5,78,100.0,1.5,12.4,6.3,6.3
5,task6,68,200.0,32.7,48.2,39.1,39.1
6,Non_RT_Task,0,400.0,36.0,192.5,600.3,600.4


In [2]:
# an additional experiment to compare the results of CFS with 1000HZ and 250HZ (on real linux system)
# FNs are the cases where the system is unschedulable with 250HZ but schedulable with 1000HZ

from functions import *
import pandas as pd
_250Hz_result_path = "../../real_linux_application/exp_results_250HZ/summary.csv"
_1000HZ_result_path = "../../real_linux_application/exp_results_CFS/summary.csv"
    
df_250 = pd.read_csv(_250Hz_result_path, sep=",")
df_1000 = pd.read_csv(_1000HZ_result_path, sep=",")

# preprocess the data
df_250 = df_250[df_250['tasksetIndex'] < 10]
df_250.columns = ['numCores',  'numTasks',  'utilization', 'tasksetIndex', '250HZ_schedulability']
df_1000 = df_1000[df_1000['tasksetIndex'] < 10]
df_1000.columns = ['numCores',  'numTasks',  'utilization', 'tasksetIndex', '1000HZ_schedulability']
merged_df = pd.merge(df_250, df_1000, on=['numCores',  'numTasks',  'utilization', 'tasksetIndex'])
merged_df = merged_df.drop(columns=['numCores'])

results = []
configurations = df_250[['numTasks', 'utilization']].drop_duplicates()
for i, (num_tasks, utilization) in configurations.iterrows():
        subset = merged_df[(merged_df['numTasks'] == num_tasks) & (merged_df['utilization'] == utilization)]
        
        # Calculate confusion matrix for the subset
        confusion = confusion_matrix(subset['250HZ_schedulability'], subset['1000HZ_schedulability'])
        
        # Calculate True Positives (TP), True Negatives (TN), False Positives (FP), and False Negatives (FN)
        if (confusion.shape == (2, 2)):
            TP = confusion[1, 1]
            TN = confusion[0, 0]
            FP = confusion[0, 1]
            FN = confusion[1, 0]
        else:
            assert confusion.shape == (1, 1), "need to check the confusion matrix : confusion.shape" + str(confusion.shape)
            if subset['250HZ_schedulability'].iloc[0] == False:
                TP = 0
                TN = confusion[0, 0]
                FP = 0
                FN = 0
            else:
                TP = confusion[0, 0]
                TN = 0
                FP = 0
                FN = 0
                
        accuracy = (TP + TN) / (TP + TN + FP + FN)
        # precision = TP / (TP + FP) if (TP + FP) != 0 else 0
        # recall = TP / (TP + FN) if (TP + FN) != 0 else 0

        # Append the results to the list
        results.append([num_tasks, utilization, TP, TN, FP, FN, accuracy])

results_df = pd.DataFrame(results, columns=['numTasks', 'utilization', 'TP', 'TN', 'FP', 'FN', 'accuracy'])
results_df = results_df.sort_values(['numTasks', 'utilization'])
print(results_df)
total_accuracy = results_df['accuracy'].mean()
print("total accuracy : ", total_accuracy)


    numTasks  utilization  TP  TN  FP  FN  accuracy
0        3.0          0.4  10   0   0   0       1.0
1        3.0          0.6  10   0   0   0       1.0
2        3.0          0.8  10   0   0   0       1.0
3        6.0          0.4  10   0   0   0       1.0
4        6.0          0.6  10   0   0   0       1.0
5        6.0          0.8  10   0   0   0       1.0
6        9.0          0.4  10   0   0   0       1.0
7        9.0          0.6  10   0   0   0       1.0
8        9.0          0.8  10   0   0   0       1.0
9       12.0          0.4  10   0   0   0       1.0
10      12.0          0.6   8   0   2   0       0.8
11      12.0          0.8   6   1   3   0       0.7
total accuracy :  0.9583333333333334
